In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
from keras import losses


2024-10-03 21:13:59.534822: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-03 21:13:59.542641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 21:13:59.551667: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 21:13:59.554350: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 21:13:59.561597: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1727979240.478308    3353 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727979240.505373    3353 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727979240.505404    3353 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
raw = open('data2.txt', mode='r', encoding='utf-8').readlines()
data = []
for line in raw:
    if line != '\n' and 'Глава' not in line:
        data.append(' '.join(line.split()[1:]))
    
data = [line.replace('\n', ' ').replace('\xa0', ' ') for line in data]
text = ' '.join(data)

In [4]:
text[:100]

'завет ЗАВЕТ Моисея  В начале сотворил Бог небо и землю. Земля же была безвидна и пуста, и тьма над б'

In [5]:
def get_features_target(seq):
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 128

alphabet = np.array(sorted(set(text)))
sym_to_idx = {}
idx_to_sym = {}

for idx, sym in enumerate(alphabet):
    sym_to_idx[sym] = idx
    idx_to_sym[idx] = sym
    
text_idx = np.array([sym_to_idx[char] for char in text])
sequences = Dataset.from_tensor_slices(text_idx).batch(BATCH_SIZE, drop_remainder=True)

dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

I0000 00:00:1727979240.905613    3353 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727979240.905660    3353 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727979240.905672    3353 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727979241.335082    3353 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727979241.335126    3353 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-03

In [6]:
model = keras.Sequential([
    keras.layers.Embedding(len(alphabet), 128),
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
    keras.layers.Dense(len(alphabet), activation='relu')
])
model.summary()

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=10, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


2024-10-03 21:14:12.783285: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


243/243 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.4769 - loss: 2.4866
Epoch 2/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9575 - loss: 0.2693
Epoch 3/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9628 - loss: 0.1907
Epoch 4/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9686 - loss: 0.1530
Epoch 5/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9705 - loss: 0.1393
Epoch 6/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9706 - loss: 0.1364
Epoch 7/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9707 - loss: 0.1349
Epoch 8/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9707 - loss: 0.1339
Epoch 9/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9708 - loss: 0.1332
Epoch 10/10
243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9709 - loss: 0.1327


In [7]:
def predict_next(sample, model, tokenizer, vocabulary, n_next, rnd_power, batch_size):
    sample_token = [tokenizer[char] for char in sample]
    predicted = sample_token

    sample_tensor = tf.expand_dims(sample_token, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    
    for _ in range(n_next):
        cur = model(sample_tensor)
        cur = cur[0].numpy() / rnd_power
        cur = tf.random.categorical(cur, num_samples=1)[-1, 0].numpy()
        predicted.append(cur)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([cur], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    res = [vocabulary[i] for i in predicted]
    generated = ''.join(res)
    return generated

In [8]:
print(predict_next(
    sample='б',
    model=model,
    tokenizer=sym_to_idx,
    vocabulary=idx_to_sym,
    n_next=200,
    rnd_power=0.6,
    batch_size=BATCH_SIZE
))

бы пру ппретогогого у во гответвы ву ни прыни ни но и м пя, вогуто но ро вя ве о стеспопро нани на и весо де прете ве бе вея вась мосто в нога вобетестостотетотете пе вера по ни во зность вого вогоппе 


In [9]:
print(predict_next(
    sample='1',
    model=model,
    tokenizer=sym_to_idx,
    vocabulary=idx_to_sym,
    n_next=100,
    rnd_power=0.2,
    batch_size=BATCH_SIZE
))

1Ве во ве весто ни во вето во веть во во во сто во во сто ва во ве ве ве вето ве во ско по весто сте 
